### Define feature extractor

In [68]:
def gender_features(word):
    return {'last_letter': word[-1:]}

### Create list of names

In [69]:
from nltk.corpus import names
import random
import nltk

names=([(name, 'male') for name in names.words('male.txt')]+
       [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

### Create our feature and test sets and classify the data

In [70]:
train_names,devtest_names, test_names = names[:6901], names[6901:7402], names[7402:7903]
train_set = [(gender_features(n), g) for (n,g) in train_names]
devtest_set = [(gender_features(n), g) for (n,g) in devtest_names]
test_set = [(gender_features(n), g) for (n,g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [71]:
classifier.classify(gender_features('Avery'))

'female'

### Now lets test the accuracy

In [72]:
print(nltk.classify.accuracy(classifier, test_set))

0.7844311377245509


### The model is about 76% accurate

### Rebuilding the gender features function to incorporate the last 2 letetrs in a persons name

In [73]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

### Re train the set

In [74]:
train_set = [(gender_features(n), g) for (n,g) in train_names]
devtest_set = [(gender_features(n), g) for (n,g) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.7445109780439122


### Our accuracy improved by 4%! Lets store the errors in a list to examine them more closely

In [75]:
errors = []
for (name,tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag,guess,name))
for (tag, guess, name) in sorted(errors):  # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct={:<8} guess={:<8} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Abagail                       
correct=female   guess=male     name=Alix                          
correct=female   guess=male     name=Arden                         
correct=female   guess=male     name=Arleen                        
correct=female   guess=male     name=Aubrey                        
correct=female   guess=male     name=Austin                        
correct=female   guess=male     name=Averil                        
correct=female   guess=male     name=Bamby                         
correct=female   guess=male     name=Barry                         
correct=female   guess=male     name=Bell                          
correct=female   guess=male     name=Bess                          
correct=female   guess=male     name=Bill                          
correct=female   guess=male     name=Blair                         
correct=female   guess=male     name=Brittney                      
correct=female   guess=male     name=Brook      

### Lets see if the number of vowels has effect on the gender of the name

In [76]:
def count_vowels(name):
    return sum(1 for letter in name if letter.lower() in 'aeiou')

def gender_features(name):
    return {
        'suffix2': name[-2:],  
        'num_vowels': count_vowels(name) 
    }

train_set = [(gender_features(n), g) for (n, g) in train_names]
devtest_set = [(gender_features(n), g) for (n, g) in devtest_names]
test_set = [(gender_features(n), g) for (n, g) in test_names]

classifier = nltk.NaiveBayesClassifier.train(train_set)

devtest_accuracy = nltk.classify.accuracy(classifier, devtest_set)
print(f'Dev-Test Set Accuracy: {devtest_accuracy}')

Dev-Test Set Accuracy: 0.7524950099800399


### Now that we have seen a 1% improvment, lets evaluate the classifier on the test set

In [77]:
test_accuracy = nltk.classify.accuracy(classifier, test_set)
print(f'Test Set Accuracy: {test_accuracy}')

Test Set Accuracy: 0.7904191616766467


### Seeing that the test set is less acurate than the Dev test-set, lets perform cross validation to perform multiple evaluations on different test sets

In [78]:
from nltk.corpus import names  
from nltk.classify.util import accuracy

male_names = names.words('male.txt')
female_names = names.words('female.txt')
labeled_male_names = [(name, 'male') for name in male_names]
labeled_female_names = [(name, 'female') for name in female_names]

labeled_names = labeled_male_names + labeled_female_names
random.shuffle(labeled_names)

data = [(gender_features(n), g) for (n, g) in labeled_names]
def cross_validate(data, folds=5):
    subset_size = len(data) // folds
    accuracy_scores = []
    
    for i in range(folds):
        test_data = data[i*subset_size:(i+1)*subset_size]
        train_data = data[:i*subset_size] + data[(i+1)*subset_size:]
        
        classifier = nltk.NaiveBayesClassifier.train(train_data)
        accuracy_scores.append(accuracy(classifier, test_data))
    
    return sum(accuracy_scores) / len(accuracy_scores)


average_accuracy = cross_validate(data, folds=5)
print(f'Average Cross-Validation Accuracy: {average_accuracy}')

Average Cross-Validation Accuracy: 0.7821158690176324
